In [105]:
%load_ext autoreload
%autoreload 2
# Built-in modules
import os
import copy
import gzip
import shutil
from pathlib import Path
from datetime import datetime, timedelta

# Basics of Python data handling and visualization
import fs
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm.auto import tqdm
from shapely import wkt
from sentinelhub import bbox_to_dimensions
from eolearn.io.local_io import ExportToTiff

# Imports from eo-learn and sentinelhub-py
from eolearn.core import EOPatch, EOTask, LinearWorkflow, FeatureType

# Visualisation utils
import sys; sys.path.append('..')
from air_quality_and_health_challenge.utils import (get_extent, 
                   draw_outline, 
                   draw_bbox, 
                   draw_feature, 
                   draw_true_color,
                   unzip_file,
                   load_tiffs,
                   days_to_datetimes,
                   datetimes_to_days,
                   reproject_tiff,
                   upscale_tiff,
                   mask_tiff)

from data.validation_dates import validation_dates
from data.test_dates import test_dates
from lib.postprocessing import postprocess
from algorithms.ensemble import Ensemble

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
ROOT = Path('./')

#data_dir = ROOT/'data/val/'
data_dir = ROOT/'data/test/'

write_dir = ROOT / 'wdir'
OUTDIR = ROOT/'submissions/final_test_submission'
os.makedirs(outdir, exist_ok=True)
os.makedirs(write_dir, exist_ok=True)

In [152]:
from configs.utils import load_config
configs = {}
configs["pm25_italy"] = load_config("configs/pm25_italy.yaml")
configs["pm25_california"] = load_config("configs/pm25_california.yaml")
configs["pm25_southafrica"] = load_config("configs/pm25_southafrica.yaml")
configs["no2_italy"] = load_config("configs/no2_italy.yaml")
configs["no2_california"] = load_config("configs/no2_california.yaml")
configs["no2_southafrica"] = load_config("configs/no2_southafrica.yaml")

In [153]:
bbox_ita = gpd.read_file(ROOT/'data/AOIs_bboxes/Italy/North_Italy_test.shp')
bbox_cal = gpd.read_file(ROOT/'data/AOIs_bboxes/California/California_test.shp')
bbox_sou = gpd.read_file(ROOT/'data/AOIs_bboxes/SouthAfrica/South_Africa_test.shp')

In [154]:
#dates = validation_dates
dates = test_dates

In [180]:
# PM2.5 SUBMISSION
for aoi, bbox in [('Italy', bbox_ita), ('California', bbox_cal), ('SouthAfrica', bbox_sou)]:
    bbox.to_crs(epsg=4326, inplace=True)
    bbox.to_file(write_dir/'bbox-wgs84.shp', driver='ESRI Shapefile')
    
    if aoi == "SouthAfrica":
        dir = data_dir/"South_Africa"
    else:
        dir = data_dir/aoi
    indir = dir/'CAMS/PM2_5'
    
    config = configs["pm25_" + aoi.lower()]
    ensemble = Ensemble(config, dir, write_dir/aoi)
    ensemble.reload()
    
        
    outdir = OUTDIR/aoi/"PM2.5"
    os.makedirs(outdir, exist_ok=True)
    for day, hour in dates[aoi]["PM2.5"]:
        infile = [f for f in os.listdir(indir) if f"day{day}_{hour}" in f][0]
        outfile = f"{day}_PM25_{aoi}.tif"
        in_eop = load_tiffs(indir, (FeatureType.DATA, 'PM2_5'), 
                        filename=infile, data_source="cams") 
        in_bbox = (in_eop.bbox.min_x, in_eop.bbox.min_y, in_eop.bbox.max_x, in_eop.bbox.max_y)
        in_data = in_eop.data["PM2_5"].squeeze()
        
        target_resolution = 1000 if aoi == "Italy" else 10_000
        target_size = bbox_to_dimensions(in_eop.bbox, target_resolution)[::-1]
        predictions = ensemble.predict((day, hour), target_size, verbose=False)
        
        out_data = postprocess(in_data, in_bbox, predictions, check=True)
        out_eop = copy.deepcopy(in_eop)
        out_eop.data["PM2_5"] = out_data.reshape(1, target_size[0], target_size[1], 1)
        
        ft = FeatureType("data")
        eo_exporter = ExportToTiff(ft, outdir/outfile, no_data_value=-1e+12)
        eo_exporter.execute(out_eop)
        out_image = mask_tiff(write_dir/'bbox-wgs84.shp', outdir/outfile, outdir/outfile)     
        
print('Done')

/home/schapke/projects/competitions/AI4EO/lib/postprocessing.py:43: RuntimeWarning: divide by zero encountered in true_divide
  predictions[pos_mask] = predictions[pos_mask] / rate[pos_mask] / 2


Done


In [181]:
# NO2 SUBMISSION
for aoi, bbox in [('Italy', bbox_ita), ('California', bbox_cal), ('SouthAfrica', bbox_sou)]:
    bbox.to_crs(epsg=4326, inplace=True)
    bbox.to_file(write_dir/'bbox-wgs84.shp', driver='ESRI Shapefile')
    
    if aoi == "SouthAfrica":
        dir = data_dir/"South_Africa"
    else:
        dir = data_dir/aoi
    indir = dir/'sentinel5P/NO2'
    
    ensemble = Ensemble(configs["no2_" + aoi.lower()], dir, write_dir/aoi)
    ensemble.reload()
    
    outdir = OUTDIR/aoi/"NO2"
    os.makedirs(outdir, exist_ok=True)
    for day, hour in dates[aoi]["NO2"]:
        infile = [f for f in os.listdir(indir) if f"day{day}_T{hour[1:]}" in f][0]
        outfile = f"{day}_NO2_{aoi}.tif"
        in_eop = load_tiffs(indir, (FeatureType.DATA, 'NO2'), 
                        filename=infile)
        no2, qa = in_eop.data["NO2"][..., 0], in_eop.data["NO2"][..., 1]
        no2[qa < 0.5] = no2[qa >= 0.5].mean()
        in_eop.data["NO2"] = no2.reshape(*no2.shape, 1)
        in_bbox = (in_eop.bbox.min_x, in_eop.bbox.min_y, in_eop.bbox.max_x, in_eop.bbox.max_y)
        in_data = in_eop.data["NO2"].squeeze()
        
        target_resolution = 1000 
        target_size = bbox_to_dimensions(in_eop.bbox, target_resolution)[::-1]
        predictions = ensemble.predict((day, hour), target_size, verbose=False)
        
        out_data = postprocess(in_data, in_bbox, predictions, check=True)
        out_eop = copy.deepcopy(in_eop)
        out_eop.data["NO2"] = out_data.reshape(1, target_size[0], target_size[1], 1)
        
        ft = FeatureType("data")
        eo_exporter = ExportToTiff(ft, outdir/outfile, no_data_value=-1e+12)
        eo_exporter.execute(out_eop)
        out_image = mask_tiff(write_dir/'bbox-wgs84.shp', outdir/outfile, outdir/outfile)     

print("Done!")

Done!


In [1]:
out_data = postprocess(in_data, in_bbox, predictions, check=True)
plt.figure(figsize=(18, 6))
plt.subplot(131, title="Model estimation of the difference from the native image")
plt.imshow(predictions)
plt.subplot(132, title="Native image")
plt.imshow(in_data)
plt.subplot(133, title="Final image")
plt.imshow(out_data)